Pedro Ivo Vasconcelos - TP

## **Capítulo 3**

1) **a.** Equações:

y1(k) = 0.7770*y1(k-1) + 0.6177*u(k-1); 

y2(k) = 0.8521*y2(k-1) + 0.6234*u(k-1) -0.4327*u(k-3) + 0.2190*u(k-4);

y3(k) = 1.4042*y3(k-1) -0.4983*y3(k-2) +0.3569*u(k-1) -0.5219*u(k-4) +0.4256*u(k-5);

y4(k) = 0.8888*y4(k-1) +0.5221*u(k-1) -0.4247*u(k-4) +0.2106*u(k-5);

y5(k) = 1.3316*y5(k-1) -0.4182*y5(k-2) +0.2399*u(k-1);


**b.** TF:
Y1(z)/U(z) = 0,6177/(z-0,7770)

Y2(z)/U(z) = (0,6234z^3 – 0,4327z^1 + 0,2190) / (z^4 - 0,8521z^3)

Y3(z)/U(z) = (0,3569z^4 – 0,5219z^1 + 0,4256) / (z^5 -1,4042z^4+0,4983z^3)

Y4(z)/U(z) = (0,5221z^4 – 0,4247z^1 + 0,2106) / (z^5 - 0,8888z^4)

Y5(z)/U(z) = (0,2399)/(z^2 -1,3316z + 0,4182)


2) O modelo passa a não atender às especificações do controlador, uma vez 
que seu comportamento passa a ser explicitamente não linear. 
A linearização é definida, inerentemente, sobre um ponto de operação, 
e se este for inadequado, a aproximação linear fica discrepante do modelo real.


3) $$ \frac{Ys}{Us}=\frac{2,77}{1,62s+1} $$ 
 
Para obter o modelo, foi obtido o valor final do G5(z), 
com isso, o ganho foi obtido, para a função de transferência, foi obtido um 
tempo aproximado de 63% da saturação, e este foi considerado tau.